In [1]:
# Import Dependencies
import requests
import pprint
import pandas as pd
# Import the EIA API key
from project_config import EIA_api_key

### Generator Report API Pull - January 2020 to January 2023

In [85]:
# Define the API key and endpoint URL
api_key = EIA_api_key
count = 0
json_array = []

# Loop to pull API results given 5000 results limit
for i in range(0,12):
    offset = 5000 * count
    url = f"https://api.eia.gov/v2/electricity/operating-generator-capacity/data/?api_key={api_key}&frequency=monthly&data[0]=latitude&data[1]=longitude&facets[stateid][]=TX&start=2020-01&end=2023-01&sort[0][column]=period&sort[0][direction]=desc&sort[1][column]=entityid&sort[1][direction]=asc&sort[2][column]=plantid&sort[2][direction]=asc&sort[3][column]=generatorid&sort[3][direction]=asc&offset={offset}&length=5000"

    # Send the API request
    response = requests.get(url)
    response.raise_for_status()

    # Retrieve the data
    data = response.json()
    data_clean = data["response"]["data"]
    for item in data_clean:
        json_array.append(item)
    count = count + 1
        


In [86]:
# Create Dataframe of Generator data
generator_full_df = pd.json_normalize(json_array)
print("Rows in Dataframe: ", len(generator_full_df))
generator_full_df.head()

Rows in Dataframe:  56365


,period,stateid,stateName,sector,sectorName,entityid,entityName,plantid,plantName,generatorid,...,energy_source_code,energy-source-desc,prime_mover_code,balancing_authority_code,balancing-authority-name,status,statusDescription,latitude,longitude,unit
0,2023-01,TX,Texas,ipp-non-chp,IPP Non-CHP,313,"Wolf Hollow I Power, LLC",55139,Wolf Hollow I LP,CTG1,...,NG,Natural Gas,CT,None,None,OP,Operating,32.33422,-97.731686,BLK1
1,2023-01,TX,Texas,ipp-non-chp,IPP Non-CHP,313,"Wolf Hollow I Power, LLC",55139,Wolf Hollow I LP,CTG2,...,NG,Natural Gas,CT,None,None,OP,Operating,32.33422,-97.731686,BLK1
2,2023-01,TX,Texas,ipp-non-chp,IPP Non-CHP,313,"Wolf Hollow I Power, LLC",55139,Wolf Hollow I LP,ST,...,NG,Natural Gas,CA,None,None,OP,Operating,32.33422,-97.731686,BLK1
3,2023-01,TX,Texas,industrial-chp,Industrial CHP,327,Air Liquide Large Industries U S LP,10298,Bayou Cogen Plant,GT1,...,NG,Natural Gas,GT,None,None,OP,Operating,29.62250,-95.045800,None
4,2023-01,TX,Texas,industrial-chp,Industrial CHP,327,Air Liquide Large Industries U S LP,10298,Bayou Cogen Plant,GT2,...,NG,Natural Gas,GT,None,None,OP,Operating,29.62250,-95.045800,None


In [88]:
# Confirm there are no duplicates in dataset as a result of for loop
generator_updated_df = generator_full_df.drop_duplicates()
print("Rows in Dataframe: ", len(generator_updated_df))
generator_updated_df.head()

Rows in Dataframe:  56365


,period,stateid,stateName,sector,sectorName,entityid,entityName,plantid,plantName,generatorid,...,energy_source_code,energy-source-desc,prime_mover_code,balancing_authority_code,balancing-authority-name,status,statusDescription,latitude,longitude,unit
0,2023-01,TX,Texas,ipp-non-chp,IPP Non-CHP,313,"Wolf Hollow I Power, LLC",55139,Wolf Hollow I LP,CTG1,...,NG,Natural Gas,CT,None,None,OP,Operating,32.33422,-97.731686,BLK1
1,2023-01,TX,Texas,ipp-non-chp,IPP Non-CHP,313,"Wolf Hollow I Power, LLC",55139,Wolf Hollow I LP,CTG2,...,NG,Natural Gas,CT,None,None,OP,Operating,32.33422,-97.731686,BLK1
2,2023-01,TX,Texas,ipp-non-chp,IPP Non-CHP,313,"Wolf Hollow I Power, LLC",55139,Wolf Hollow I LP,ST,...,NG,Natural Gas,CA,None,None,OP,Operating,32.33422,-97.731686,BLK1
3,2023-01,TX,Texas,industrial-chp,Industrial CHP,327,Air Liquide Large Industries U S LP,10298,Bayou Cogen Plant,GT1,...,NG,Natural Gas,GT,None,None,OP,Operating,29.62250,-95.045800,None
4,2023-01,TX,Texas,industrial-chp,Industrial CHP,327,Air Liquide Large Industries U S LP,10298,Bayou Cogen Plant,GT2,...,NG,Natural Gas,GT,None,None,OP,Operating,29.62250,-95.045800,None


In [53]:
# Create Sector Dataframe to store sectorID and name
sector_df = generator_full_df[["sector", "sectorName"]].drop_duplicates(ignore_index=True)
sector_df = sector_df.set_index("sector")
sector_df

,sectorName
sector,
electric-utility,Electric Utility
ipp-non-chp,IPP Non-CHP
industrial-non-chp,Industrial Non-CHP
industrial-chp,Industrial CHP
commercial-non-chp,Commercial Non-CHP
ipp-chp,IPP CHP
commercial-chp,Commercial CHP


In [54]:
# Create Entity Dataframe to store entityID and name
entity_df = generator_full_df[["entityid", "entityName"]].drop_duplicates(ignore_index=True)
entity_df = entity_df.set_index("entityid")
entity_df

,entityName
entityid,
1015,Austin Energy
12501,Midlothian Energy LLC
61944,MN8 Energy LLC
60155,"Old Settler Wind, LLC"
60146,Ameresco Federal Solutions
...,...
58420,Harbor Wind Tenant LLC
63713,CatanSolar
63884,Oklaunion Power Station


In [55]:
# Create Plant Dataframe to store plantID and name
plant_df = generator_full_df[["plantid", "plantName"]].drop_duplicates(ignore_index=True)
plant_df = plant_df.set_index("plantid")
plant_df

,plantName
plantid,
3548,Decker Creek
55091,Midlothian Energy Facility
61895,Toyota HQ Plan
58488,"Bryan Solar, LLC"
60366,Old Settler Wind
...,...
10187,Engineered Carbons Echo Cogeneration
127,Oklaunion
3477,Lone Star


In [91]:
# Create State Dataframe to store statusID and name
state_df = generator_full_df[["stateid", "stateName"]].drop_duplicates(ignore_index=True)
state_df = state_df.set_index("stateid")
state_df

,stateName
stateid,
TX,Texas


In [56]:
# Create Status Dataframe to store statusID and description
status_df = generator_full_df[["status", "statusDescription"]].drop_duplicates(ignore_index=True)
status_df = status_df.set_index("status")
status_df

,statusDescription
status,
OP,Operating
OS,Out of service and NOT expected to return to s...
OA,Out of service but expected to return to servi...
SB,Standby/Backup: available for service but not ...


In [96]:
# Create Energy Source Dataframe to store statusID and description
sources_df = generator_full_df[["energy_source_code", "energy-source-desc"]].drop_duplicates(ignore_index=True).sort_values(by=["energy_source_code"])
sources_df = sources_df.set_index("energy_source_code")
sources_df

,energy-source-desc
energy_source_code,
AB,Agriculture Byproducts
BFG,Blast-Furnace Gas
BLQ,Black Liquor
DFO,Disillate Fuel Oil
LFG,Landfill Gas
LIG,Lignite
MWH,Electricity used for energy storage
NG,Natural Gas
NUC,Nuclear


In [60]:
# Confirm whether this column is blank for all rows - if blank for all, can delete from dataframe
balancing_authorities = generator_full_df["balancing-authority-name"].unique()
balancing_authorities

array([None,
       'Midcontinent Independent Transmission System Operator, Inc..',
       'Electric Reliability Council of Texas, Inc.',
       'Southwest Power Pool', 'El Paso Electric Company',
       'Public Service Company of New Mexico'], dtype=object)

In [58]:
# Drop columns to create summarized generator dataframe
generators_clean_df = generator_full_df.drop(columns=["stateName", "sectorName", "entityName", "plantName", "statusDescription", "energy-source-desc", "unit"])
generators_clean_df.head()

,period,stateid,sector,entityid,plantid,generatorid,technology,energy_source_code,prime_mover_code,balancing_authority_code,balancing-authority-name,status,latitude,longitude
0,2023-01,TX,electric-utility,1015,3548,GT1,Natural Gas Fired Combustion Turbine,NG,GT,None,None,OP,30.3033,-97.6128
1,2023-01,TX,ipp-non-chp,12501,55091,STK6,Natural Gas Fired Combined Cycle,NG,CS,None,None,OP,32.4302,-97.0537
2,2023-01,TX,ipp-non-chp,12501,55091,STK5,Natural Gas Fired Combined Cycle,NG,CS,None,None,OP,32.4302,-97.0537
3,2023-01,TX,ipp-non-chp,12501,55091,STK4,Natural Gas Fired Combined Cycle,NG,CS,None,None,OP,32.4302,-97.0537
4,2023-01,TX,ipp-non-chp,12501,55091,STK3,Natural Gas Fired Combined Cycle,NG,CS,None,None,OP,32.4302,-97.0537


In [97]:
# Export all Dataframes to CSV for importing into SQL
sector_df.to_csv("Output/Sector_DF.csv")
entity_df.to_csv("Output/Entity_DF.csv")
plant_df.to_csv("Output/Plant_DF.csv")
status_df.to_csv("Output/Status_DF.csv")
state_df.to_csv("Output/States_DF.csv")
sources_df.to_csv("Output/Energy_Sources_DF.csv")
generators_clean_df.to_csv("Output/Generators_DF.csv")

### Price Details API Pull - January 2020 to January 2023

In [62]:
# Price Details
url = f"https://api.eia.gov/v2/natural-gas/pri/sum/data/?api_key={api_key}&frequency=monthly&data[0]=value&facets[duoarea][]=STX&start=2020-01&end=2023-01&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000"
response = requests.get(url)
response.raise_for_status()

# Retrieve the data
price_data = response.json()
pprint.pprint(price_data)

{'apiVersion': '2.1.4',
 'request': {'command': '/v2/natural-gas/pri/sum/data/',
             'params': {'api_key': 'msLbwqq4bJqp1UAFRX7ulbpXsXoju1uJalc0DDJz',
                        'data': ['value'],
                        'end': '2023-01',
                        'facets': {'duoarea': ['STX']},
                        'frequency': 'monthly',
                        'length': 5000,
                        'offset': 0,
                        'sort': [{'column': 'period', 'direction': 'desc'}],
                        'start': '2020-01'}},
 'response': {'data': [{'area-name': 'TEXAS',
                        'duoarea': 'STX',
                        'period': '2022-12',
                        'process': 'PEU',
                        'process-name': 'Electric Power Price',
                        'product': 'EPG0',
                        'product-name': 'Natural Gas',
                        'series': 'N3045TX3',
                        'series-description': 'Texas Natural Gas Pri

In [63]:
# Create Dataframe of Price data
price_data_df = pd.json_normalize(price_data["response"]["data"])
print("Rows in Dataframe: ", len(price_data_df))
price_data_df.head()
# Note there are fewer rows than it states on the API dashboard because the final 2 pages of API results are blank

Rows in Dataframe:  288


,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2022-12,STX,TEXAS,EPG0,Natural Gas,PEU,Electric Power Price,N3045TX3,Texas Natural Gas Price Sold to Electric Power...,5.34,$/MCF
1,2022-12,STX,TEXAS,EPG0,Natural Gas,PRS,Price Delivered to Residential Consumers,N3010TX3,Texas Price of Natural Gas Delivered to Reside...,16.01,$/MCF
2,2022-12,STX,TEXAS,EPG0,Natural Gas,VRX,Percent Sold to The Commercial Sector,NA1504_STX_4,Texas Natural Gas % of Total Residential - Sal...,99.90,%
3,2022-12,STX,TEXAS,EPG0,Natural Gas,VFA,% of Industrial Consumers - Sales,N3035TX4,Percent of Industrial Natural Gas Deliveries i...,28.60,%
4,2022-12,STX,TEXAS,EPG0,Natural Gas,VFC,Percent Sold to The Residential Consumers,N3020TX4,Percent of Commercial Natural Gas Deliveries i...,75.90,%


In [64]:
# Create Series Dataframe to store seriesID and description
series_df = price_data_df[["series", "series-description"]].drop_duplicates(ignore_index=True)
series_df

,series,series-description
0,N3045TX3,Texas Natural Gas Price Sold to Electric Power...
1,N3010TX3,Texas Price of Natural Gas Delivered to Reside...
2,NA1504_STX_4,Texas Natural Gas % of Total Residential - Sal...
3,N3035TX4,Percent of Industrial Natural Gas Deliveries i...
4,N3020TX4,Percent of Commercial Natural Gas Deliveries i...
5,N3020TX3,Texas Price of Natural Gas Sold to Commercial ...
6,N3050TX3,Natural Gas Citygate Price in Texas (Dollars p...
7,N3035TX3,Texas Natural Gas Industrial Price (Dollars pe...


In [65]:
# Create Process Dataframe to store processID and name
process_df = price_data_df[["process", "process-name"]].drop_duplicates(ignore_index=True)
process_df

,process,process-name
0,PEU,Electric Power Price
1,PRS,Price Delivered to Residential Consumers
2,VRX,Percent Sold to The Commercial Sector
3,VFA,% of Industrial Consumers - Sales
4,VFC,Percent Sold to The Residential Consumers
5,PCS,Price Delivered to Commercial Sectors
6,PG1,City Gate Price
7,PIN,Industrial Price


In [89]:
# Drop duplicate or irrelevant columns to create clean price dataframe
price_df_clean = price_data_df.drop(columns=["duoarea", "product", "process", "series-description"])
price_df_clean.head()

,period,area-name,product-name,process-name,series,value,units
0,2022-12,TEXAS,Natural Gas,Electric Power Price,N3045TX3,5.34,$/MCF
1,2022-12,TEXAS,Natural Gas,Price Delivered to Residential Consumers,N3010TX3,16.01,$/MCF
2,2022-12,TEXAS,Natural Gas,Percent Sold to The Commercial Sector,NA1504_STX_4,99.90,%
3,2022-12,TEXAS,Natural Gas,% of Industrial Consumers - Sales,N3035TX4,28.60,%
4,2022-12,TEXAS,Natural Gas,Percent Sold to The Residential Consumers,N3020TX4,75.90,%


In [90]:
# Drop rows with % as unit, as we're only looking at price data, and delete blank rows
natural_gas_price_data_df = price_df_clean[price_df_clean.units != "%"].dropna()
natural_gas_price_data_df

,period,area-name,product-name,process-name,series,value,units
0,2022-12,TEXAS,Natural Gas,Electric Power Price,N3045TX3,5.34,$/MCF
1,2022-12,TEXAS,Natural Gas,Price Delivered to Residential Consumers,N3010TX3,16.01,$/MCF
5,2022-12,TEXAS,Natural Gas,Price Delivered to Commercial Sectors,N3020TX3,12.33,$/MCF
6,2022-12,TEXAS,Natural Gas,City Gate Price,N3050TX3,6.82,$/MCF
7,2022-12,TEXAS,Natural Gas,Industrial Price,N3035TX3,6.52,$/MCF
...,...,...,...,...,...,...,...
281,2020-01,TEXAS,Natural Gas,Price Delivered to Commercial Sectors,N3020TX3,5.57,$/MCF
283,2020-01,TEXAS,Natural Gas,Industrial Price,N3035TX3,2.32,$/MCF
284,2020-01,TEXAS,Natural Gas,Price Delivered to Residential Consumers,N3010TX3,8.26,$/MCF
285,2020-01,TEXAS,Natural Gas,Electric Power Price,N3045TX3,1.93,$/MCF


In [69]:
# Export all Dataframes to CSV for importing into SQL
series_df.to_csv("Output/Series_DF.csv")
process_df.to_csv("Output/Processes_DF.csv")
natural_gas_price_data_df.to_csv("Output/Energy_Prices_DF.csv")
price_data_df.to_csv("Output/Prices_Full_report.csv")